In [1]:
import pandas as pd
import datetime
from datetime import timedelta
import time
import json
import pytz

from azure.servicebus import ServiceBusClient, ServiceBusMessage
from azure.cosmos import CosmosClient
import azure.cosmos.cosmos_client as cosmos_client

from pyspark.sql.types import StructType, ArrayType  
from pyspark.sql.functions import col, when
from delta.tables import *
from pyspark.sql.functions import *
import numpy as np
starttime = time.time()


storage_account_name = ""
container_name = ""


pd.set_option('display.max_rows', 100)

ModuleNotFoundError: No module named 'azure.servicebus'

In [ ]:
file_location = f"/2021/05/10/"
dbutils.fs.ls(file_location)

In [ ]:
hour_list = ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
date = datetime.datetime.now(pytz.timezone('Asia/Shanghai')) - timedelta(days=1)
year = date.year
month = date.month
day = date.day
whole_df = pd.DataFrame()
for i in hour_list:
  df = spark.read.parquet(f"2021/05/10/{i}")
  print(f"2021/05/10/{i}")

  df_ = df.drop('Version','UnitType','Country_Code','GatewayID','SubID','SIMCardID','MotionSensorLink','CSQ','BatteryVoltage','BackupStatus','LiftState','UnitPowerOff','Floor','Direction','DoorZone','FrontDoorStatus','RearDoorStatus','CountCallUp','CountCallDown','Speed','Load','CarDoorLock','FloorDoorLock','SafetyCircuitStatus','DoorProtectorStatus','PassengerExists','DetailDataPerSecond','BlinkMessage')
  df_ = df_.toPandas()
  whole_df = whole_df.append(df_)  
  whole_df.sort_values('ReportTime',inplace=True)
df = whole_df
whole_df

In [ ]:
abnormal_df = df[df['UnitNumber'].isin(df[df['OPMode'] == 'NAV']['UnitNumber'].drop_duplicates())]
abnormal_df['OPMode_shift'] = abnormal_df.groupby('UnitNumber')['OPMode'].shift(1)
abnormal_df

In [ ]:
result = pd.DataFrame()
for num , elevator in enumerate(abnormal_df['UnitNumber'].drop_duplicates()):
  start = abnormal_df[(abnormal_df['OPMode'] == 'NAV' )& (abnormal_df['OPMode_shift'] != 'NAV' )& (abnormal_df['UnitNumber'] == elevator )]
  start = start['ReportTime'].to_list()
  
  end = abnormal_df[(abnormal_df['OPMode'] != 'NAV' )& (abnormal_df['OPMode_shift'] == 'NAV' )& (abnormal_df['UnitNumber'] == elevator )]
  end = end['ReportTime'].to_list()
  
  var = []
  for start_time in start:
    for end_time_index in range(len(end)):
      if end[end_time_index] > start_time:
        var.append(end[end_time_index] - start_time)
        end = end[end_time_index:]
        break
      else:
        continue
  print(num)
  diction = {'0-22':0,'22-42':0,'42-62':0,'62-120':0,'120以上':0}
  for value in var:
    if value <= 22:
      diction['0-22'] += 1
    elif 22 < value <= 42:
      diction['22-42'] += 1
    elif 42 < value <= 62:
      diction['42-62'] += 1
    elif 62 < value <= 120:
      diction['62-120'] += 1
    elif value > 120:
      diction['120以上'] += 1
  diction = pd.DataFrame(diction,index=[elevator])
  result = result.append(diction)
result

In [ ]:
result.loc['Row_sum'] = result.apply(lambda x: x.sum())
final = pd.DataFrame(result.loc['Row_sum']).T
final.insert(0,'Time','2020/05/12')
final

In [ ]:
result.to_csv('/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/result.csv')

In [ ]:
abnormal_df[(abnormal_df['OPMode'] == 'NAV' )& (abnormal_df['OPMode_shift'] != 'NAV' )]  #正常到不正常

abnormal_df[(abnormal_df['OPMode'] != 'NAV' )& (abnormal_df['OPMode_shift'] == 'NAV' )]

In [ ]:
ls /dbfs/FileStore/shared_uploads/lilian.lu@otis.com/

In [ ]:
Abnormal = ["NAV", "DTC", "DTO","BKF","BSS","COR",'DAR','DBF','DLF','DLM','EPC','EPR','EPW','EQR','ESB','FLD','HAD','INI','LPS','LRN','NAV','OOS','RCY','REC','STP','UFS','PFO']

In [ ]:
# df = spark.read.parquet(f"dbfs:/mnt/testmount/OtisOne/robustel/elevatorheartbeat/2021/05/11/00")
# df_ = df.drop('Version','UnitType','Country_Code','GatewayID','SubID','SIMCardID','MotionSensorLink','CSQ','BatteryVoltage','BackupStatus','LiftState','UnitPowerOff','Floor','Direction','DoorZone','FrontDoorStatus','RearDoorStatus','CountCallUp','CountCallDown','Speed','Load','CarDoorLock','FloorDoorLock','SafetyCircuitStatus','DoorProtectorStatus','PassengerExists','DetailDataPerSecond','BlinkMessage')
# df = df_.toPandas()
# df.sort_values('ReportTime',inplace=True)
df = whole_df

In [ ]:
abnormal_df = df[df['UnitNumber'].isin(df[df['OPMode'] == 'NAV']['UnitNumber'].drop_duplicates())]
abnormal_df

In [ ]:
abnormal_df['UnitNumber'].drop_duplicates()

In [ ]:
abnormal_df['OPMode_shift'] = abnormal_df.groupby('UnitNumber')['OPMode'].shift(1)
abnormal_df

In [ ]:
final = []
for num , elevator in enumerate(abnormal_df['UnitNumber'].drop_duplicates()):
  print(num)
  elevator_data = df[df['UnitNumber'] == elevator]
  elevator_data['ReportTime_shift'] = elevator_data['ReportTime'].shift(1)
  elevator_data['ReportTime_shift'] = elevator_data['ReportTime'] - elevator_data['ReportTime_shift']
  
  # 统计上传
  normal_count , abnormal_count = 0 , 0 
  for time in elevator_data['ReportTime_shift'].tolist():
    if time < 30:
      normal_count += 1
    elif time >= 30:
      abnormal_count += 1
  max_report_time = elevator_data.loc[:,"ReportTime_shift"].max()
  
  
  
  #统计异常
  normal_list = []
  for count , opmode in enumerate(elevator_data['OPMode'].to_list()):
    if opmode not in Abnormal:
      normal_list.append(count)
  if len(normal_list) == 0:
    new_diction = {'异常次数大于7':1}
  else:
    diction = {}
    for i in range(len(normal_list)-1):
      if normal_list[i+1] - normal_list[i] != 1:
        if normal_list[i+1] - normal_list[i] - 1 not in diction.keys():
          diction[normal_list[i+1] - normal_list[i] -1 ] = 1
        else:
          diction[normal_list[i+1] - normal_list[i] -1 ] += 1
    count_bigger_than_7 = 0
    for i in {k : v for k, v in diction.items() if k>=7}.values():
      count_bigger_than_7 += 1
    new_diction = {k : v for k, v in diction.items() if k < 7}
    new_diction['异常次数大于7'] =  count_bigger_than_7
  diction = pd.DataFrame(new_diction,index=[0])
  diction['正常时间上传次数'] , diction['断线次数'], diction['最大上传时间'] = normal_count  , abnormal_count , max_report_time
  diction.index = [elevator]
  if len(final) == 0 :
    final = diction
  else:
    final = pd.concat([final,diction])
final

In [ ]:
len(df['UnitNumber'].drop_duplicates())

In [ ]:
a = ['NOR','NAV','NAV','NOR','NAV','NOR','NAV','NAV','NAV','NOR']
normal_list = []
# for count , opmode in enumerate(a):
for count , opmode in enumerate(elevator_data['OPMode'].to_list()):
  if opmode not in Abnormal:
      normal_list.append(count)
if len(normal_list) == 0:
  diction = {'一直异常':1}
else:
  diction = {}
  for i in range(len(normal_list)-1):
    if normal_list[i+1] - normal_list[i] != 1:
      if normal_list[i+1] - normal_list[i] - 1 not in diction.keys():
        diction[normal_list[i+1] - normal_list[i] -1 ] = 1
      else:
        diction[normal_list[i+1] - normal_list[i] -1 ] += 1
print(diction)
count_bigger_than_7 = 0
for i in {k : v for k, v in diction.items() if k>=7}.values():
  count_bigger_than_7 += 1
new_diction = {k : v for k, v in diction.items() if k < 7}
new_diction['值大于7'] =  count_bigger_than_7
new_diction

In [ ]:
diction = pd.DataFrame(diction,index=[0])
diction

In [ ]:
diction.index = ['F8N24E89']
diction

In [ ]:
diction2 = diction.copy()
diction2['4'] = 1
final = pd.concat([diction,diction2])
final

In [ ]:
if final_dataframe not exist

In [ ]:

Normal = ["NOR", "IDL", "PRK", ""]

In [ ]:
hour_list = ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
date = datetime.datetime.now(pytz.timezone('Asia/Shanghai')) - timedelta(days=1)
year = date.year
month = date.month
day = date.day
whole_df = pd.DataFrame()
for i in hour_list:
  if int(day) <= 9:
    df = spark.read.parquet(f"dbfs:/mnt/testmount/OtisOne/robustel/elevatorheartbeat/{year}/0{month}/0{day}/{i}")
  else:
    df = spark.read.parquet(f"dbfs:/mnt/testmount/OtisOne/robustel/elevatorheartbeat/{year}/0{month}/{day}/{i}")
  df_ = df.drop('Version','UnitType','Country_Code','GatewayID','SubID','SIMCardID','MotionSensorLink','CSQ','BatteryVoltage','BackupStatus','OPMode','LiftState','UnitPowerOff','Floor','Direction','DoorZone','FrontDoorStatus','RearDoorStatus','CountCallUp','CountCallDown','Speed','Load','CarDoorLock','FloorDoorLock','SafetyCircuitStatus','DoorProtectorStatus','PassengerExists','DetailDataPerSecond','BlinkMessage')
  df_ = df_.toPandas()
  whole_df = whole_df.append(df_)  
whole_df[['ReportTime','ControllerLink']] = whole_df[['ReportTime','ControllerLink']].astype(float)